In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
import collections
import numpy as np
from spacy.en import English
import string
import unicodedata
import gensim
import re
from collections import Counter
from itertools import chain
from scipy.spatial import distance
import spacy
from collections import Counter

nlp = spacy.load('en')
parser = English()


sqd_data = pd.read_json("/home/anjana/Downloads/train-v1.1.json") 


In [2]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/home/anjana/Downloads/GoogleNews-vectors-negative300.bin', binary=True)


In [3]:
stopword = stopwords.words('english')
qtntype_list = ['why','how', 'what', 'when','which', 'who','whom', 'where','whose']
pun_list = list(string.punctuation)
delelements_list = stopword + qtntype_list + pun_list

In [4]:
def replace_all(repls, str):
   # return re.sub('|'.join(repls.keys()), lambda k: repls[k.group(0)], str)                                    
   return re.sub('|'.join(re.escape(key) for key in repls.keys()),
                 lambda k: repls[k.group(0)], str)

In [5]:
def preprocess_question(para_question):
    question_list = []
    for i in range(len(para_question)):
        l1 = para_question[i]['question']
        l1 = unicodedata.normalize('NFKD', l1).encode('ascii','ignore').decode('utf-8')
        question_list.append(l1)
    
    #Appropriate question type replacement
    Question_list = []
    for i in range(len(question_list)):
        Question_list.append(replace_all({"how": "how", "when": "when","what": "what","whom": "whom","which": "which", "where": "where", "who":"who" ,"whose":"whose"}, question_list[i]))
        
    return Question_list

In [6]:
def preprocess_text(para_context):

    context = unicodedata.normalize('NFKD', para_context).encode('ascii','ignore').decode('utf-8')
    #print(context)
    #sentence_tokeniztion
    senttokenized_context = sent_tokenize(context)
    sentencelist = []
    for sent_token in senttokenized_context:
        sentencelist.append(sent_token) 
    
    sentence_list = []
    lemmatizer = WordNetLemmatizer()


    for i in sentencelist:
        words = nltk.word_tokenize(i)
        words = ([x.lower() for x in words])
        words = ([x for x in words if x not in delelements_list])
        words = [lemmatizer.lemmatize(x) for x in words]
        words = [[s.encode('ascii').decode('utf-8') for s in words]]
        sentence_list.append(words)
    return sentencelist, sentence_list

In [7]:
def question_type_extractor(question_mapping):
    Question_mapping = []
    for i in range(len(question_mapping)):
        if question_mapping[i] != 'how many' and question_mapping[i] != 'how much':
            Question_mapping.append(question_mapping[i].split()[0])
        else:
             Question_mapping.append(question_mapping[i])
    return Question_mapping


In [8]:
def wmd_que_ans_pair(que_list, sen_list):
    wmd_matrix = np.zeros([len(que_list),len(sen_list)]) 
    for i in range(len(que_list)):
        for j in range(len(sen_list)):
            wmd_matrix[i,j] =  word2vec_model.wmdistance(que_list[i],sen_list[j])  
    wmd_df = pd.DataFrame(wmd_matrix)
    wmd_df['sentence'] = wmd_df.T.idxmin()
    wmd_df['qnno'] = wmd_df.index
    que_no_ans_sen_no = []
    for i in range(len(wmd_df)):
        que_no_ans_sen_no.append([wmd_df['qnno'][i], wmd_df['sentence'][i]])
    qn_sentence = []
    for i in range(len(que_no_ans_sen_no)):
        if que_no_ans_sen_no[i][0] == que_list.index(que_list[i]):
            qn_sentence.append(que_list[i])
    an_sentence = []
    for i in range(len(que_no_ans_sen_no)):
        for j in range(len(sen_list)):
            if que_no_ans_sen_no[i][1] == sen_list.index(sen_list[j]):
                an_sentence.append(sen_list[j]) 
    return qn_sentence, an_sentence

In [9]:
def que_ner_ans_ner(que_sentence, ans_sentence):
    ans_sentence_mapping = []
    for i in range(len(ans_sentence)):
        doc_ans = nlp(ans_sentence[i])
        for ent in doc_ans.ents:
            ans_sentence_mapping.append([i,ent.label_])

    que_sentence_mapping = []
    for i in range(len(que_sentence)):
        doc_que = nlp(que_sentence[i])
        for ent in doc_que.ents:
            que_sentence_mapping.append([i,ent.label_])
    return que_sentence_mapping, ans_sentence_mapping

In [10]:
def que_df_ans_df(que_sen_map, ans_sen_map):
    qno_anstag_df = pd.DataFrame(ans_sen_map,columns = ['QuestionNo','AnswerTag'])
    qno_quetag_df = pd.DataFrame(que_sen_map, columns = ['QuestionNo', 'QuestionTag'])
    return qno_quetag_df, qno_anstag_df



In [11]:
def que_ans_combo_(questionword, answerword):
    ques_ans_combo = []
    for i in range(len(questionword)):
        for j in range(len(answerword)):
            ques_ans_combo.append([questionword[i],answerword[j]])
    return(ques_ans_combo)


In [12]:
def question_type(process_que_list):
    question_mapping = [] 
    for i in range(len(process_que_list)):
        l2 = nltk.word_tokenize(process_que_list[i])
        #print(len(l2))
        #print(l2)
        missed = l2[-1]        
        if missed == '?':
            for j in range(len(l2)):


                if l2[j] in qtntype_list:
                    combine_list = l2[j]+" "+l2[j+1]
                    question_mapping.append(combine_list)

        else:

             for k in range(len(l2)):#8

                if k < len(l2)-1:  #7        

                    if l2[k] in qtntype_list:
                        combine_list = l2[k]+" "+l2[k+1]
                        question_mapping.append(combine_list) 
                else:

                        if l2[k] in qtntype_list:
                            combine_list = l2[k]+" "+'abc'
                            question_mapping.append(combine_list) 
    return question_mapping


In [13]:
def qntypefun(qnlist):
    punc=set(",./;'?&-")

    common_list = []
    for i in qnlist:
        list1 = (i.lower().split())
        list2 = list(set(list1).intersection(qtntype_list)) 
        for j in list2:
            strp =''.join(c for c in j if not c in punc)
            common_list.append(strp) 
        if list2 == []:
            common_list.append('others')
        
        
    return common_list

In [14]:
co_occ_matrix_df = pd.read_csv("/home/anjana/co_occ_matrix_final", header=0, index_col=0)
co_occ_matrix = co_occ_matrix_df.values
co_occ_matrix

array([[  385,   360,    12,    81,     8,    29,    30,    21,   421,
           26,    13,    11,    65,   230,   577,   589],
       [ 3047,  2574,   259,   774,    76,   234,   264,   113,  4630,
          224,   722,   653,   777,  6862,  5670,  5829],
       [19765, 15133,  1416,  4367,   435,  1246,  1979,   791, 18870,
          622,  2112,   827,  3896, 12288, 28041, 31681],
       [ 3206,  2266,   182,   570,    58,   257,   302,    84,  4718,
          102,    56,    84,   810,  1659,  4272,  4559],
       [ 1959,  1188,   291,   536,    35,   105,   140,    40,  1371,
           31,    39,    35,   389,   947,  3366,  2944],
       [ 8106,  3404,   230,   673,    90,   286,   513,   102,  4063,
           92,    88,   102,   955,  1903,  5250,  6888],
       [ 3886,  2290,   338,   835,    92,   199,   369,   104,  2779,
           84,   145,   130,   671,  1749,  4917,  5023],
       [  201,    79,     8,    13,     1,    13,     4,     3,   111,
            1,     1,     

In [16]:
row_list = ['why','how','what','when','where','who', 'which','whom','whose','others','PERSON',
          'NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
          'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE', 'ORG']


column_list = ['PERSON','NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
               'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE','ORG']

In [17]:
co_occ_matrix_df['Tags'] = co_occ_matrix_df.index

from sklearn.preprocessing import normalize
normed_matrix = normalize(co_occ_matrix, axis=1, norm='l1') 
normed_matrix_df = pd.DataFrame(normed_matrix, index = row_list, columns= column_list)


/home/anjana/nltk/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [18]:
normed_matrix_df['Tags'] = normed_matrix_df.index

#normalized heatmap
melt_normed_matrix_df = pd.melt(normed_matrix_df,id_vars = ['Tags'])
melt_normed_matrix_df.head()

,Tags,variable,value
0,why,PERSON,0.134710
1,how,PERSON,0.093158
2,what,PERSON,0.137765
3,when,PERSON,0.138279
4,where,PERSON,0.146020


In [19]:
new_normed_matrix  = 1 - normed_matrix
new_normed_matrix_df = pd.DataFrame(new_normed_matrix, index = row_list, columns= column_list)
new_normed_matrix_df['Tags'] = new_normed_matrix_df.index

In [20]:
melt_new_normed_matrix_df = pd.melt(new_normed_matrix_df,id_vars = ['Tags'])
melt_new_normed_matrix_df.head()

,Tags,variable,value
0,why,PERSON,0.865290
1,how,PERSON,0.906842
2,what,PERSON,0.862235
3,when,PERSON,0.861721
4,where,PERSON,0.853980


In [21]:
answer_dict = {'LOC': 1,'PERSON': 2,'NORP': 3,'FAC': 4,'PRODUCT': 5,'EVENT': 6,'WORK_OF_ART': 7,'LANGUAGE': 8,'DATE':9,'TIME': 10,'PERCENT': 11,'MONEY': 12,'ORDINAL': 13,'CARDINAL': 14,'GPE': 15, 'ORG': 16}
question_dict = {'LOC': 10,
                 'PERSON': 20,
                 'NORP': 30,
                 'FAC': 40,
                 'PRODUCT': 50,
                 'EVENT':60,
                 'WORK_OF_ART':70,
                 'LANGUAGE':80,
                 'DATE': 90,
                 'TIME': 100,
                 'PERCENT': 110,
                 'MONEY': 120,
                 'ORDINAL':130,
                 'CARDINAL': 140,
                 'GPE': 150,
                 'ORG': 160,
                 
                 'what': 170,
                 'when': 180,
                 'where': 190,
                 'who': 200,
                 'which':210,
                 'whom': 220,
                'whose': 230,
                'how':240,
                'why':250,
                'others':260}

In [22]:
melt_new_normed_matrix_df_1 = melt_new_normed_matrix_df.copy()
melt_new_normed_matrix_df_1["Tags"].replace(question_dict, inplace=True)
melt_new_normed_matrix_df_1['variable'].replace(answer_dict, inplace = True)

In [23]:
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy, Reader
reader_new = Reader(line_format='user item rating', sep='\t', rating_scale=(0,1))

In [24]:
csv_file_1 = melt_new_normed_matrix_df_1.to_csv("new_melt.txt", sep= '\t', header = False, index = False)

In [25]:
reader_new = Reader(line_format='user item rating', sep='\t', rating_scale=(0,1))
melt_1 = Dataset.load_from_file("/home/anjana/new_melt.txt", reader_new)
print(melt_1)

In [26]:
import math
from sklearn.metrics import mean_squared_error

In [27]:
reconstruction_error_1 = []
for i in range(20):
    
    algo = SVDpp(n_factors=i)
    
    for trainset, testset in melt_1.folds():
        algo.train(trainset)
        predictions_svdpp = algo.test(testset)
        
    
    reconstruct = np.dot(algo.pu, algo.qi.T)
    rmse = mean_squared_error(new_normed_matrix, reconstruct)**0.5
    reconstruction_error_1.append(rmse)
    print(i, rmse)

0 0.94047868429
1 0.940502403557
2 0.939969831526
3 0.940617119208
4 0.940254271648
5 0.940037478996
6 0.940585729432
7 0.941123111524
8 0.942635811164
9 0.940156000007
10 0.940718699098
11 0.942493776307
12 0.943147165273
13 0.94124648593
14 0.941427995474
15 0.938654270509
16 0.939956597446
17 0.93720426279
18 0.942888098849
19 0.941068892567


In [28]:
algo = SVDpp(n_factors=10)
    
for trainset, testset in melt_1.folds():
    algo.train(trainset)
    predictions_svdpp = algo.test(testset)
    svdpp_df = pd.DataFrame(predictions_svdpp)

In [29]:
svdpp_df['error'] = svdpp_df.r_ui - svdpp_df.est #error =  observed - estimated
svdpp_df.head()

,uid,iid,r_ui,est,details,error
0,230,5,0.996004,1.000000,{'was_impossible': False},-0.003996
1,90,14,0.890518,0.937567,{'was_impossible': False},-0.047050
2,100,4,0.994778,0.976835,{'was_impossible': False},0.017943
3,70,11,0.995025,1.000000,{'was_impossible': False},-0.004975
4,130,8,0.995697,0.945261,{'was_impossible': False},0.050435


In [30]:
reconstruct = np.dot(algo.pu, algo.qi.T) #reconstructed matrix
rmse = mean_squared_error(new_normed_matrix, reconstruct)**0.5

In [31]:
reconstruct_error = new_normed_matrix - reconstruct
reconstruct_error_df = pd.DataFrame(reconstruct_error,index = row_list,columns = column_list)


In [32]:
row_column_list =['why',
 
  'how',
 'whose',               
 'what',
 'when',
 'where',
 'who',
 'which',
 'whom',
'others',
                
                  
 'q_PERSON',
 'q_NORP',
 'q_FAC',
 'q_LOC',
 'q_PRODUCT',
 'q_EVENT',
 'q_WORK_OF_ART',
 'q_LANGUAGE',
 'q_DATE',
 'q_TIME',
 'q_PERCENT',
 'q_MONEY',
 'q_ORDINAL',
 'q_CARDINAL',
 'q_GPE',
 'q_ORG',
 'a_PERSON',
 'a_NORP',
 'a_FAC',
 'a_LOC',
 'a_PRODUCT',
 'a_EVENT',
 'a_WORK_OF_ART',
 'a_LANGUAGE',
 'a_DATE',
 'a_TIME',
 'a_PERCENT',
 'a_MONEY',
 'a_ORDINAL',
 'a_CARDINAL',
 'a_GPE',
 'a_ORG']

In [33]:
qn_ans_list = list(algo.pu) + list(algo.qi)
qn_ans_list

qn_ans_list_copy = qn_ans_list.copy()
len(qn_ans_list_copy)

42

In [34]:
question_answer_dis = np.zeros((len(row_column_list), len(row_column_list))) 

for i in range(len(qn_ans_list)):
    for j in range(len(qn_ans_list_copy)):
        question_answer_dis[i,j] = distance.euclidean(qn_ans_list[i],qn_ans_list_copy[j])

In [35]:
eucli_matrix_df = pd.DataFrame(question_answer_dis, index = row_column_list, columns= row_column_list)
eucli_matrix_df

,why,how,whose,what,when,where,who,which,whom,others,...,a_WORK_OF_ART,a_LANGUAGE,a_DATE,a_TIME,a_PERCENT,a_MONEY,a_ORDINAL,a_CARDINAL,a_GPE,a_ORG
why,0.000000,0.369390,0.363307,0.283515,0.368420,0.370810,0.317100,0.260210,0.343792,0.259073,...,0.391371,0.374754,0.328225,0.297465,0.336358,0.310173,0.274145,0.332458,0.357638,0.261070
how,0.369390,0.000000,0.360829,0.465410,0.489510,0.289396,0.322319,0.455112,0.410958,0.314226,...,0.445512,0.387252,0.544785,0.459130,0.497970,0.425309,0.426977,0.493475,0.411387,0.421921
whose,0.363307,0.360829,0.000000,0.573273,0.484266,0.402513,0.434900,0.415099,0.386007,0.401284,...,0.412578,0.507149,0.595443,0.521874,0.580396,0.543716,0.324952,0.454619,0.434054,0.357629
what,0.283515,0.465410,0.573273,0.000000,0.490411,0.526640,0.436833,0.352533,0.510801,0.305893,...,0.562173,0.351014,0.359026,0.375087,0.320718,0.363193,0.440069,0.468581,0.506385,0.445194
when,0.368420,0.489510,0.484266,0.490411,0.000000,0.404092,0.310591,0.381433,0.454696,0.369421,...,0.472915,0.414894,0.499270,0.231219,0.316996,0.532049,0.367383,0.294792,0.375615,0.342236
where,0.370810,0.289396,0.402513,0.526640,0.404092,0.000000,0.251512,0.433129,0.400759,0.391974,...,0.442014,0.437673,0.492225,0.389495,0.507826,0.490856,0.373436,0.437228,0.491103,0.340949
who,0.317100,0.322319,0.434900,0.436833,0.310591,0.251512,0.000000,0.307938,0.282247,0.295941,...,0.436145,0.399608,0.378503,0.294022,0.397334,0.413007,0.399324,0.262623,0.418334,0.316930
which,0.260210,0.455112,0.415099,0.352533,0.381433,0.433129,0.307938,0.000000,0.387187,0.359945,...,0.494234,0.403046,0.293318,0.359397,0.372175,0.429679,0.346755,0.217538,0.456953,0.264499
whom,0.343792,0.410958,0.386007,0.510801,0.454696,0.400759,0.282247,0.387187,0.000000,0.362601,...,0.472407,0.550300,0.471550,0.404236,0.532491,0.463114,0.425777,0.330136,0.444332,0.436777
others,0.259073,0.314226,0.401284,0.305893,0.369421,0.391974,0.295941,0.359945,0.362601,0.000000,...,0.400821,0.340487,0.417669,0.354442,0.355809,0.341493,0.359588,0.400024,0.397146,0.334882


In [36]:
def word_count(doc_ner_list):
    count = Counter(doc_ner_list)
    map_count = []
    for i in count.items():
        map_count.append(i)
    return map_count

def bag_of_words(doc_ner_list, unique_ner):
    dis_matrix = np.zeros(len(unique_ner), dtype = float)
    nbow = word_count(doc_ner_list)
   
    doc_len = len(doc_ner_list)
    for i, freq in nbow:
        dis_matrix[unique_ner.index(i)] = freq / float(doc_len)  # Normalized word frequencies.
    return dis_matrix


In [37]:
def ner_combination(map_que_ner, map_ans_ner):
     
    unique_removed_new_ner = list(set(map_que_ner + map_ans_ner))
    return unique_removed_new_ner

In [38]:
from pyemd import emd


In [39]:
def qntypefun(qnlist):
    punc=set(",./;'?&-")

    common_list = []
    for i in qnlist:
        list1 = (i.lower().split())
        list2 = list(set(list1).intersection(qtntype_list)) 
        for j in list2:
            strp =''.join(c for c in j if not c in punc)
            common_list.append(strp) 
        if list2 == []:
            common_list.append('others')
        
        
    return common_list

###### Tag distance ######

In [43]:
len(sqd_data)

442

In [44]:
Tagdistance = [] 

for i in range(len(sqd_data)):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 

    for k in range(len(paragraph_pointer)):#no of paragraphs in a row of the df

        
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])

        

        ls = [] 

        que_type_clean = qntypefun(processed_que_list)

        for m in range(len(que_type_clean)):

            map_que_ner = []
            map_que_ner.append(que_type_clean[m])
            #print("Que_NER", " ", map_que_ner)
            for j in range(len(processed_sen_list)):



                map_ans_ner = []
                ans_ner = nlp(processed_sen_list[j])
                for ent in ans_ner.ents:
                    map_ans_ner.append('a_' + ent.label_)
                #print("Ans_NER", " ", map_ans_ner)
                unique_ner = ner_combination(map_que_ner, map_ans_ner)
                #print("Unique_NER", " ", unique_ner)
                #print()
                # distance_matrix = np.zeros((len(unique_ner), len(unique_ner)))
                d_1 = bag_of_words(map_que_ner, unique_ner)
                #print(d_1)

                d_2 = bag_of_words(map_ans_ner, unique_ner)
                #print(d_2)
                sub_matrix_df = eucli_matrix_df.loc[(unique_ner),(unique_ner)]
                #print(sub_matrix_df)
                sub_matrix= sub_matrix_df.as_matrix()
                sub_matrix[np.isnan(sub_matrix)] = 0

                #print(sub_matrix)
                score = emd(d_1, d_2, sub_matrix)
                #print(processed_sen_list[j])
                #print('Score = ', score)
                #l1 = [m,j,score] 
                #ls.append(pd.DataFrame([m,j,score])
                #tagdis = pd.DataFrame(l1) 
                #tagdis['TagRank'] = tagdis.groupby('Qnno')['Tagdis'].rank(method = "first",ascending=True)
                #Tagdistance.append(ls)
                ls.append([m,j,score]) 
                
                
                
                df = pd.DataFrame(ls,columns = ['Qnno','Sentno','TagDistance'])
                
                df['Rank'] = df.groupby('Qnno')['TagDistance'].rank(method = "first",ascending=True)

        Tagdistance.append(df)
    
    #return Tagdistance
                
                #print([i,k,m,j,score])
                
               # print("****************************************************************************************************")



In [46]:
len(Tagdistance)

18896

In [47]:
Tagdistance[0].head()

,Qnno,Sentno,TagDistance,Rank
0,0,0,0.378392,4.0
1,0,1,0.349630,1.0
2,0,2,0.416877,7.0
3,0,3,0.378679,5.0
4,0,4,0.371880,3.0


##### W2V distance ##### 

In [57]:
w2vdistance = [] 

for i in range(len(sqd_data)):
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 
    

    for k in range(len(paragraph_pointer)):
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[k]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[k]['qas'])
        distance = [] 
        for m in range(len(processed_que_list)):
            for j in range(len(processed_sen_list)):

                #word2vec_model.init_sims(replace=True)
                score = word2vec_model.wmdistance(processed_que_list[m].split(), processed_sen_list[j].split())
                score_df = [m,j,score]
                distance.append(score_df)
                distance_df = pd.DataFrame(distance,columns = ['Qnno','Sentno','w2vDistance'])
                distance_df['Rank'] = distance_df.groupby('Qnno')['w2vDistance'].rank(method = "first",ascending=True)
        w2vdistance.append(distance_df)

        


In [58]:
len(w2vdistance)

18896

In [60]:
w2vdistance[0].head()

,Qnno,Sentno,w2vDistance,Rank
0,0,0,2.652292,3.0
1,0,1,2.631091,2.0
2,0,2,2.906310,7.0
3,0,3,2.734407,5.0
4,0,4,2.763077,6.0


In [61]:
def rankfun(df):
    return list(df[df['Rank']==1.0]['Sentno']) 

In [62]:
tagfirstrank = [] 
w2vfirstrank = [] 

for i in range(len(Tagdistance)):
    tag = rankfun(Tagdistance[i])
    w2v = rankfun(w2vdistance[i])
    tagfirstrank.append(tag)
    w2vfirstrank.append(w2v)
    
    

In [64]:
len(w2vfirstrank)


18896

###### Qn-Actual ans ##### 

In [68]:
def answerprocessfun(context):
    answer_ls = [] 
    for i in range(len(context['qas'])):
        #ls1 = []
        #
       # ls1.append(context['qas'][i]['answers'][0]['text'].lower().split())
        answer_ls.append(context['qas'][i]['answers'][0]['text'].lower().split())
    return answer_ls


In [69]:
def commonsen(list1,list2):
    return len(set(list1) & set(list2))


In [70]:
qn_anssent = [] 

for b in range(len(sqd_data)):
    #Content_list.append(b)                                 
    
    #Qn_list = [] 
    #Sentence_list = [] 
    paragraph_pointer = sqd_data.data[b]['paragraphs'] 


    for j in range(len(paragraph_pointer)):
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[j]['context']) 
        #processed_que_list = preprocess_question_second(paragraph_pointer[j]['qas'])
        que_list = preprocess_question(paragraph_pointer[j]['qas'])
        processed_ans_list = answerprocessfun(paragraph_pointer[j])

        qn_ans_commonwords_list = np.zeros([len(processed_ans_list),len(process_sen_tokenized_list)])
        for i in range(len(processed_ans_list)):
                for j in range(len(process_sen_tokenized_list)):
                    qn_ans_commonwords_list[i,j] = len(set(processed_ans_list[i]) & set(process_sen_tokenized_list[j][0]))
                df = pd.DataFrame(qn_ans_commonwords_list)
                df['AnsSent'] = df.T.idxmax()
                df['Qnno'] = df.index 
        qn_anssent.append(df[['Qnno','AnsSent']])

In [71]:
len(qn_anssent)

18896

In [72]:
actual_anssent = [] 
for i in range(len(qn_anssent)):
    actual_anssent.append(list(qn_anssent[i]['AnsSent'])) 

In [73]:
actual_anssent[0]

[5, 2, 1, 4, 1]

In [74]:
def zeronemapping(list1,list2):
    mappinglist = [] 
    for i in range(len(list1)):
        if (list1[i]==list2[i]):
            mappinglist.append(1)
        else:
            mappinglist.append(0)
    return mappinglist

In [75]:
tag_mapping = [] 
w2v_mapping = [] 
for i in range(len(w2vfirstrank)):
    w2v_mapping.append(zeronemapping(actual_anssent[i],w2vfirstrank[i])) 
    tag_mapping.append(zeronemapping(actual_anssent[i],tagfirstrank[i])) 

In [76]:
print(len(w2v_mapping))
print(len(tag_mapping))

18896
18896


###### Total Sentence number ###### 

In [77]:
qn_actualans_df = pd.concat(qn_anssent)
totalsen_no = len(qn_actualans_df)
totalsen_no

87599

In [78]:
w2v_accuracy = (sum([sum(l) for l in w2v_mapping])/totalsen_no)*100
tag_accuracy = (sum([sum(l) for l in tag_mapping])/totalsen_no)*100

In [80]:
print("w2v_distance accuracy", " :", w2v_accuracy,"%")
print("tag_distance accuracy",":",tag_accuracy,"%") 

w2v_distance accuracy  : 60.427630452402425 %
tag_distance accuracy : 22.247970867247343 %
